<h1>DAVE's INSIDES</h1>
Must run to work

In [10]:
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch.utils.data import DataLoader, TensorDataset

# Load the MNIST dataset
from torchvision.datasets import MNIST
from torch.utils.tensorboard import SummaryWriter
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Create the neural network
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #Fully Connected
        # self.fc1 = torch.nn.Linear(28*28, 128)
        # self.fc2 = torch.nn.Linear(128, 256)
        # self.fc3 = torch.nn.Linear(256, 128)
        # self.fc4 = torch.nn.Linear(128, 10)

        
        self.cv = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2),
            
            torch.nn.Conv2d(in_channels=4, out_channels=16, kernel_size=5, padding=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2,2),

            torch.nn.Conv2d(in_channels=16, out_channels=64, kernel_size=5, padding=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2,2),

            torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=5, padding=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2,2),
        )

        self.fc = torch.nn.Sequential(
            torch.nn.Linear(64, 200),
            torch.nn.ReLU(),
            torch.nn.Dropout2d(0.1),
            torch.nn.Linear(200, 200),
            torch.nn.ReLU(),
            torch.nn.Dropout2d(0.1),
            torch.nn.Linear(200, 10)
        )

        
    

    def forward(self, x):
        # x = torch.nn.functional.relu(self.fc1(x))
        # x = torch.nn.functional.relu(self.fc2(x))
        # x = torch.nn.functional.relu(self.fc3(x))
        # x = self.fc4(x)
        x = self.cv(x)
        x = x.flatten(1)
        x = self.fc(x)
        
        # x = self.cv2(x)
        return x

model = Net()
model.to(device)
#model.load_state_dict(torch.load("MNISTtrainedModel.pt"))


Net(
  (cv): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(4, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(16, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=64, out_features=200, bias=True)
    (1): ReLU()
    (2): Dropout2d(p=0.1, inplace=False)
    (3): Linear(in_features=200, out_features=200, bias=True)
    (4): ReLU()
    (5): Dropout2d(p=0.1, inplace=False)
    (6): Linear

<h1>Teach DAVE (OPTIONAL)</h1>
Do not run if you have a slow computer

In [11]:
def test(epoch, writer, device, dataloader, datasetTotal):
    numCorrect = 0
    for idx, (x_batch,y_batch) in enumerate(dataloader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        outputs = model(x_batch)

        pred = y_batch[torch.arange(y_batch.shape[0], device=device),torch.argmax(outputs, dim=1)]
        numCorrect += torch.sum(pred).item()
    
    writer.add_scalar('Accuracy/test', numCorrect*100/datasetTotal, epoch)
    return numCorrect*100/datasetTotal


datasetTrain = MNIST(root='.', download=True, train=True)
datasetValidate = MNIST(root='.', download=True, train=False)
writer = SummaryWriter()

# Preprocess the data
#Lin
# X_train = dataset.train_data.numpy().reshape((-1, 28*28)) / 255.0
# X_test = dataset.test_data.numpy().reshape((-1, 28*28)) / 255.0

#Conv
X_train = datasetTrain.data.detach().cpu().numpy().reshape(-1,1,28,28) / 255.0
X_test = datasetValidate.data.detach().cpu().numpy().reshape(-1,1,28,28) / 255.0

y_train = datasetTrain.targets.detach().cpu().numpy()
y_test = datasetValidate.targets.detach().cpu().numpy()

#Image augmentation doubles data points
X_trainNoAA = np.ceil(X_train)
X_testNoAA = np.ceil(X_test)

X_train = np.concatenate((X_train, X_trainNoAA))
X_test = np.concatenate((X_test, X_testNoAA))
y_train = np.concatenate((y_train, y_train))
y_test = np.concatenate((y_test, y_test))

del X_testNoAA, X_trainNoAA
####

print(X_train.shape)
print(X_test.shape)


# Convert the labels to one-hot encoded vectors
y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]




BATCHSIZE = 1024
dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float),torch.tensor(y_train, dtype=torch.float))
dataloader = DataLoader(dataset=dataset, batch_size=BATCHSIZE, shuffle=True)

datasetValidation = TensorDataset(torch.tensor(X_test, dtype=torch.float), torch.tensor(y_test, dtype=torch.float))
dataloaderValidation = DataLoader(datasetValidation, batch_size=BATCHSIZE, shuffle=False)


# Define the loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)
numEpochs = 10

# Train the model
for epoch in range(numEpochs):
    numCorrect=0
    for idx, (x_batch,y_batch) in enumerate(dataloader):
        # Forward pass
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        outputs = model(x_batch)
        loss = loss_fn(outputs, y_batch)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #Analyse
        pred = y_batch[torch.arange(y_batch.shape[0], device=device),torch.argmax(outputs, dim=1)]
        numCorrect += torch.sum(pred).item()
    writer.add_scalar('Loss/train', loss.item(), epoch)
    writer.add_scalar('Accuracy/train', numCorrect*100/y_train.shape[0])

    testAcc = test(epoch, writer, device, dataloaderValidation, y_test.shape[0])
    print(f"Epoch: {epoch} accuracy: {numCorrect*100/y_train.shape[0]}, Test Accuracy: {testAcc}", end="\r")

torch.save(model.state_dict(), "MNISTtrainedModel.pt")
writer.close()




(120000, 1, 28, 28)
(20000, 1, 28, 28)


<h1>Load DAVE if you don't want to train one yourself!</h1>
If you want to train or modify, rename MODELTOUSE

In [3]:
MODELTOUSE = "MNISTtrainedModel.pt"
dataset = MNIST(root='.', download=True)
model = Net()
model.load_state_dict(torch.load(MODELTOUSE))
model = model.eval()
model.to(device=device)


# X_train = dataset.train_data.numpy().reshape(-1,1,28,28) / 255.0
# X_test = dataset.test_data.numpy().reshape(-1,1,28,28) / 255.0



# y_train = dataset.train_labels.numpy()
# y_test = dataset.test_labels.numpy()

# y_train = np.eye(10)[y_train]
# y_test = np.eye(10)[y_test]

# xTestTensor = torch.tensor(X_test, dtype=torch.float32, device=device)
# yTestTensor = torch.tensor(y_test, dtype=torch.float32, device=device)

# xTestPredictions = torch.argmax(model(xTestTensor),dim=1)
# # numCorrect = 0

# # for x in xTestPredictions:
# #     if y_test[x,xTestPredictions[x]] == 1:
# #         numCorrect+=1
# # print(numCorrect/yTestTensor.size(0))
# # print(numCorrect)

# numCorrect2 = torch.sum(yTestTensor[torch.arange(yTestTensor.size(0)), xTestPredictions], dtype=torch.long)
# print(f"Error rate ({yTestTensor.size(0)} Samples): {((yTestTensor.size(0)-numCorrect2.item())*100/yTestTensor.size(0))}%")





Net(
  (cv): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(4, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(16, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Dropout2d(p=0.1, inplace=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=10, bias=True

<h1>Draw your own number and let <abbr title="Deeply Accurate Vision Expert">DAVE</abbr> guess your number</h1>
<p>Draw by holding down the left mouse button</p>
<p>To reset, right click.</p>
<p>When your number is complete, press enter!</p>

In [12]:
import pygame
from tkinter import *
from tkinter import messagebox
import numpy as np
Tk().wm_withdraw()

class Grid:
    def __init__(self) -> None:
        self.cells = []
        for j in range(0,28):
            row = []
            for i in range(0,28):
                row.append(Cell([i*20, j*20]))
            self.cells.append(row)
    def draw(self):
        for row in self.cells:
            for cell in row:
                cell.draw()
    def drawOn(self, pos):
        x = pos[0]
        y = pos[1]

        #Outside boundries
        if y >= 28*20 or y < 0:
            return
        if x >= 28*20 or x < 0:
            return
        row = y//20
        column = x//20

        #Change cell over
        self.cells[row][column].drawOn()
        
        #Slightly add white to outer cells

        # if row+1 < 28:
        #     self.cells[row+1][column].bleedOn(255)

        # if row-1 >= 0:
        #     self.cells[row-1][column].bleedOn(255)

        # #Horizontal
        # if column+1 < 28:
        #         self.cells[row][column+1].bleedOn(255)
        # if column-1 >= 0:
        #         self.cells[row][column-1].bleedOn(255)

    def toNumpy(self):
        arr = np.zeros(shape=(28,28),dtype=float)
        for j in range(28):
            for i in range(28):
                v,_,_a = self.cells[j][i].value
                arr[j,i] = v/255
        return arr
    def clear(self):
        for row in self.cells:
            for cell in row:
                cell.clear()

class Cell:
    def __init__(self, pos):
        self.pos=pos
        self.width=20
        self.value = BLACK

    def draw(self):
        pygame.draw.rect(screen,self.value,[self.pos[0], self.pos[1], self.width, self.width])


    def drawOn(self):
        self.value = WHITE
    def bleedOn(self, amount):
        amount /=30
        if self.value == WHITE:
            return
        r,g,b = self.value
        r += amount
        g += amount
        b += amount
        r = min(255.0, r)
        g = min(255.0, g)
        b = min(255.0, b)
        self.value = (r,g,b)
    def clear(self):
        self.value = BLACK

# Define some colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

# Call this function so the Pygame library can initialize itself
pygame.init()
 
# Create an 560x560 sized screen
screen = pygame.display.set_mode([560, 560])
 
# This sets the name of the window
pygame.display.set_caption('Number Drawer for DAVE')
 
clock = pygame.time.Clock()
 

 
# Load and set up graphics.
grid = Grid()


done = False
 
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
        if event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 3:
                grid.clear()
        if event.type == pygame.KEYDOWN:
            if event.key == 13: #ENTER Key pressed
                #Run through NN
                npDigit = grid.toNumpy()
                digitTensor = torch.tensor(npDigit.reshape(1, 1, 28, 28), device=device, dtype=torch.float32)
                modelPred = torch.argmax(model(digitTensor)[0]).item()
                
                messagebox.showinfo("MODEL Predictions: You drew a..", str(modelPred))


                

                
    if pygame.mouse.get_pressed()[0]:
        pos=pygame.mouse.get_pos()
        grid.drawOn(pos)
        grid.drawOn([pos[0] + 20, pos[1]])
        grid.drawOn([pos[0] - 20, pos[1]])
        grid.drawOn([pos[0], pos[1]+20])
        grid.drawOn([pos[0], pos[1]-20])
    grid.draw()

 
    pygame.display.flip()
 
    clock.tick(165)
 
pygame.quit()



In [5]:
torch.cuda.empty_cache()